<!-- Auto table of contents -->
<h1 class='tocIgnore'>Générateur semi-automatique de page HTML "quizz oiseaux"</h1>
<p>(application des "Oiseaux à l'Oreille" : les quizz <b>2020</b>)</p>
<div style="overflow-y: auto">
  <h2 class='tocIgnore'>Table des matières</h2>
  <div id="toc"></div>
</div>

In [ ]:
%%javascript
$.getScript('../../ipython_notebook_toc.js')

# Imports

In [ ]:
import os
import datetime as dt

from collections import OrderedDict as odict

import requests

from IPython.display import HTML

In [ ]:
import sys
sys.path.append('..')

import quizz

import importlib as implib

In [ ]:
# Les étapes : les mêmes pour tous les quizz et tous les exercices.
KEtapes = \
[quizz.DescripteurEtape(index=1, id='lancement', titre='Les enregistrements à analyser et les 1er indices'),
 quizz.DescripteurEtape(index=2, id='indices2', titre='Quelques indices à mi-parcours'),
 quizz.DescripteurEtape(index=3, id='reponse', titre='Les réponses : analyse détaillée des enregistrements')]

# Commutateur de publication locale ou internet

In [ ]:
KDossierQuizz = '.'

pubLocale = True

dossierSons = os.path.join(KDossierQuizz, 'enregistrements')

if pubLocale:
    
    # 1) Locale
    urlDossierSons = dossierSons.replace('\\', '/') # Jouer les sons en lisant les fichiers locaux.
    urlDossierAnnexes = os.path.join(KDossierQuizz, 'fichiers').replace('\\', '/') # idem, ailleurs

else:
    
    # 2) Internet
    urlDossierSons = '.' # Jouer depuis XenoCanto si enregistrement en vient, et n'a pas eté modifié ('-extrait'),
                         # sinon depuis le dossier de la page HTML
    urlDossierAnnexes = '.' # Fichiers annexes sur mon site, dans le dossier de la page HTML

# Préambule et remerciements

In [ ]:
# texte syntaxe markdown
KPreambule = \
"""
L'été avance, les migrateurs commencent déjà à passer par chez nous ...
il est temps de reprendre les quizz d'ornithoreille !

Exactement sur le même modèle que les années précédentes, alors si vous connaissez déjà,
vous pouvez vous épargner le blabla qui suit, et passer directement au <a href="#Aout">quiz d'août</a> :-)

Chacun de ces exercices, nommé comme le mois de 2020 durant lequel il a est lancé, se déroule en *3 étapes*,
publiée chacune 3 semaines après la précédente :
1. la première, le **lancement**, vous donne accès aux enregistrements à analyser
   et à quelques premiers indices pour démarrer,
2. la deuxième, à **mi-parcours**, vous donne des indices plus conséquents, pour confirmer ou pas vos premières idées
   et / ou pour aller plus loin en ciblant quelques séquences plus difficiles,
3. la troisième et dernière, pour terminer, vous donne la "**solution**", en détaillant pour chaque enregistrement
   et chaque espèce, les différents types de chants et cris entendus, et les instants précis concernés.

Pour accéder à l'étape suivante, lorsqu'elle est publiée, il vous suffit de cliquer sur le **petit oeil vert**
<img height="16" src="{{dossierAttache}}/fa-eye-regular.svg" alt="Montrer"/>
situé sous le titre de cette dernière ; bien sûr, tant qu'elle n'est pas publiée, vous ne verrez rien de plus ;-)

Chaque quiz porte sur **plusieurs enregistrements** (en général 3) indépendants, pour varier les milieux et les espèces.

Pour chaque enregistrement (répété à chaque étape), vous trouverez :
* un **lecteur audio intégré**, pour l'écouter, le ré-écouter ... etc, autant de fois que nécessaire,
* un **lien de téléchargement**, pour pouvoir l'analyser tranquillement et précisément
sur votre tablette ou ordinateur, avec un lecteur plus pratique notamment en ce qui concerne
le positionnement temporel ; pour télécharger l'enregistrement, il vous suffit de cliquer sur le bouton
<img height="16" src="{{dossierAttache}}/fa-download-solid.svg" alt="Télécharger"/>
(à moins qu'il ne faille un clic droit, suivi de "enregistrer la cible du / le lien sous ..."),
* le nom de l'**auteur** de l'enregistrement et les détails de sa **licence** de publication,
* à la dernière étape, si disponible, le lien vers la **page du site source** de l'enregistrement
  (xeno-canto.org en général).

Pub. <a href="http://audacity.fr/" target="_blank">Audacity</a> est - entre autres - un lecteur audio très pratique
et précis pour la gestion du temps : essayez-le, je vous le conseille :-) ; il est disponible gratuitement
et en toute sécurité (logiciel Open Source) sous Windows, Mac OS ou Linux (mais pas Android ou i-OS :-(.

Chaque quiz permet à chacun de jouer **quel que soit son niveau**, du débutant au plus expérimenté :
si vous ne trouvez pas tout de A à Z, c'est **normal**, certains enregistrements sont plus difficiles
que d'autres, et à dessein : à vous, pour chaque concert, de trouver **votre défi personnel** ;
par exemple, au delà des espèces que vous avez reconnues rapidement, en choississant 1 chant (ou un cri), ou 2,
que vous ne reconnaissez pas, mais pour lequel les indices à mi-parcours vous orientent suffisamment,
et en cherchant vous-même la solution ailleurs (les <a href="http://jpmeuret.free.fr/nature.html" target="_blank">
supports de formation</a>, bien sûr, mais aussi des CD, des applis dédiées, des livres, des sites internet
comme <a href="http://www.oiseaux.net/oiseaux/france.html" target="_blank">oiseaux.net</a>,
<a href="https://www.xeno-canto.org/" target="_blank">xeno-canto.org</a>, des amis, et surtout les oiseaux réels
lors de vos ballades, dans votre jardin ...).

Ce défi personnel, que vous seul(e) pouvez ajuster à vos connaissances actuelles, vous permettra
de progresser, en vous poussant **au delà de ce que vous savez** déjà ; mais il ne devra pas être **trop ambitieux**
d'un coup, car l'important, c'est d'éviter de dépasser le niveau de frustration qui fait renoncer,
et de rester dans celui qui pousse à avancer :-)

Et puis gardez confiance : l'exercice "en intérieur" sur ordinateur n'est pas simple, tout le monde le dit :
* on n'est pas immergé dans le milieu naturel,
* la qualité du son n'est pas toujours très bonne (enregistrement, casque, haut-parleurs, ...),
* on ne peut pas tourner la tête pour mieux entendre ... même en stéréo, on manque de repères dans l'espace,
* ...

Vous constaterez vite, lors de vos ballades "dans le monde réel", avec les espèces que vous connaissez,
que vous êtes plus à l'aise à l'extérieur : meilleur son, moins de bruit de fond, plus de clarté,
plus de profondeur, ... c'est normal, aucun intermédiaire entre les oiseaux et vos oreilles !

... Mais il y a un hic : en nature, il n'y a pas de bouton "Rejouer" ;-)

Dernier détail, mais d'importance, je compte sur vous pour me les signaler les **coquilles** et **erreurs**
qui se sont inévitablement glissées dans ce qui suit, à me faire part de votre **désaccord** éventuel
sur telle ou telle affirmation ou identification, ou encore à me signaler toute chose que j'aurais "**zappée**"
dans les enregistrements :-)

Amusez-vous bien !
"""

In [ ]:
KAttribsEtMercis = \
"""
Merci aussi à <a href="https://www.faune-auvergne.org/index.php?m_id=53&id=705" target="_blank">
Monique et Jacques Lombardy</a> et Yvon Le Corre (pas le peintre, l'ornithologue, breton aussi :-)
pour leurs splendides photos de la grive musicienne et du bruant ortolan en haut de page (tous droits réservés).
"""

# Merci aussi à Annie, Camille, Dominique, Florence, Monique et Sylvain pour leurs questions et remarques
# bien utiles pour affiner les indices de mi-parcours et les listes de confusions possibles, et pour leurs trouvailles
# qui m'ont permis de creuser d'avantage les analyses et de compléter ainsi les listes d'espèces avec celles
# que j'avais zappées :-)

# Quizz d'août

In [ ]:
KAoutIntro = """
  Pour commencer, quatre concerts plutôt calmes (enfin ... le dernier moins :-),
  tous enregistrés en Auvergne, et même dans le Puy-de-Dôme ...
  histoire de nous rappeler - si besoin - ces heures de confinement du printemps 2020 ;-)
"""

In [ ]:
KAoutBocagePrairiesMoyenneAltitude = quizz.DescripteurExercice(

  index='a',
  id='BocagePrairiesMoyenneAltitude',
  titre="Bocage et prairies à moyenne altitude",
  lieu="La Batrande (est), Aydat, Puy-de-Dôme", altitude=930,
  date="2 avril 2020", heure="le matin", duree="58 s",
  auteur="Anne Citron",
  licence="<a href=\"https://creativecommons.org/licenses/by-nc-sa/4.0/deed.fr\" target=\"_blank\">CC BY-NC-SA</a>",
  milieux="""Bocage en déprise et prairies sur plateau de moyenne altitude""",
  fichierSon="200402-hhmm-AydatBatrandeE-AC.mp3",
  dossierSons=urlDossierSons, dossierAttache=urlDossierAnnexes,
    
  etapes=dict(
      
    lancement="""
      Un premier concert campagnard, clair et ensoleillé, peu chargé, sans doute assez tôt le matin
      (certains oiseaux ont encore la voix enrouée ;-).
      
      5 espèces a priori, communes à très communes, dont 3 chantent véritablement.
      """,
          
    indices2="""
      C'est une alouette qui occupe le 1er plan presque tout au long de l'enregistrement,
      avec ses jolis flûtés mélancoliques à peine interrompus ... mais laquelle ?
      
      Au 2ème plan, ou un peu plus loin,
      * un corvidé bien audible avec sa voix grave (un peu enrouée) : ne cherchez pas midi ... à 9h du matin :-)
      * tout à l'opposé de la bande sonore, dans les aigus, et un peu plus éloignée,
        une mésange égrène ses petites phrases délicates et aigrelettes, à 2 "plateaux" :
        d'abord 1 ou 2 notes identiques, puis une série de notes un peu moins aigues,
        répétées sur un rythme plus rapide (par ex. à 5.5s, 6.5s, 12.5s ... 12 fois en tout au moins :-),
      * un pic bien sûr, "ricane" de son perchoir à qq distance (vers 45s).
      
      A même distance environ, mais il faut être plus attentif/ve : entre 14-20s,
      dans les mêmes aigus que la mésange, un petit chant rythmé bien connu 
      (et peut-être à 35.5-36s plus loin et furtivement ?).
      
      Enfin, plus difficile, bien qu'au 1er plan : une et plus probablement 2 mésanges d'une autre espèce
      égrènent de-ci de-là un florilège de petits cris caractéristiques :
      * trc mats et roulés, longs (à 30s, 33s, 40s, 44s ...),
      * tc plus secs, rauques et courts (de 36 à 40s, ...),
      * petits sii divers un peu tremblotés, voire oscillants,
        courts ou un peu moins, un peu tout au long de l'enregistrement.
      """,
          
    reponse="""
      """))

HTML(KAoutBocagePrairiesMoyenneAltitude.lecteurHtml(urlDossierSons))

In [ ]:
KAoutVergerVillageChateau = quizz.DescripteurExercice(

  index='b',
  id='StAgoulinGirauds',
  titre="Un verger près du parc du château",
  lieu="Les Girauds, Saint-Agoulin, Puy-de-Dôme", altitude=500,
  date="19 avril 2020", heure="fin d'après-midi", duree="40 s",
  auteur="<a href=\"http://jpmeuret.free.fr/nature.html\" target=\"_blank\">Jean-Philippe Meuret</a>",
  licence="<a href=\"https://creativecommons.org/licenses/by-nc-sa/4.0/deed.fr\" target=\"_blank\">CC BY-NC-SA</a>",
  milieux="""Verger, jardins dans village à proximité d'un parc boisé.""",
  fichierSon="200419-1757-306-StAgoulinGirauds-JPM.mp3",
  dossierSons=urlDossierSons, dossierAttache=urlDossierAnnexes,
    
  etapes=dict(
      
    lancement="""
      Il est juste 6h du soir à l'horloge de l'église, mais les cloches qui les sonnent ne semblent pas
      perturber le moins du monde les solistes de ce concert plutôt calme et aéré ...
      pas plus que la petite pluie qui se renforce un peu à la fin.
      
      6 espèces très communes chantent, et une crie.
      
      Si vous vous ennuyez, essayez donc de compter les chanteurs de chaque espèce :-)
      """,
          
    indices2="""
      Commençons par les graves, avec les 2 colombidés qu'on s'attend à entendre dans un village ;
      rappelez-vous le nombre "hou" qui caractérise les phrases de chant de chaque espèce (sans parler de la hauteur) :
      * 1ère espèce entre 1s et 10s,
      * 2nde espèce entre 24.5s et 36s (et les "gémissements" à 18.5-20s et 39s lui appartiennent aussi :
         un des rares cris de l'espèce) ; et il y a au moins un autre individu qui chante, au loin (vers 21s par ex.).
      
      Bien sûr, les vedettes de cette scène sont plutôt les 2 turdidés qui se répondent avec calme et distinction
      (ne voyez-vous pas leurs noeuds-papillons et leur port de tête altier ?), et même invention pour le plus proche :
      * d'abord "l'autre", entre 2 et 4s,
      * ensuite le plus proche, entre 4 et 6s,
      * "l'autre", entre 15 et 17s,
      * ensuite le plus proche, entre 17 et 18s,
      * ...
      
      Et ne vous laissez pas embrouiller par les fioritures excentriques entre 33.5 et 35.5s ...
      ce sont de facéties l'individu le plus proche !
      
      Les autres espèces sont moins accessibles, car plus éloignées et moins sonores ou plus furtives,
      il faut monter le volume :
      * assez isolé à 23.5s et 32.5s, un rouge-queue lance avec énergie sa phrase "raccourcie" du soir
        (à moins que ce ne soit la distance et les échos multiples sur les murs du village),
        une phrase épurée notamment de sa fin si caractéristique et évocatrice ... il ne reste qu'un rapide
        "sisisi" très dynamique,
      * le parc du chateau a de beaux arbres, le fringille en question ne peut faire autrement que d'en être
        (à 18-20s, et aussi 3.5-5.5s),
      * et cette mésange n'a pas le choix non plus, même si elle ne s'exécute qu'à regret, en râlant :
        un cris assez long vers 11-12s,
      * enfin, il manque bien sûr une espèce dans cet environnement de village rural, avec ses fermes
        et ses vieux murs accueillants, une espèce des plus sociables (enfin ... par petits groupes
        au printemps) : en fond sonore lointain tout au long de l'enregistrement, mais moins noyés
        dans le reste du concert dans la 1ère seconde, et entre 27 et 32s par ex.
    """,
          
    reponse="""
      """))

HTML(KAoutVergerVillageChateau.lecteurHtml(urlDossierSons))

In [ ]:
KAoutGrandesCulturesLimagnePerchee = quizz.DescripteurExercice(

  index='c',
  id='GrandesCulturesLimagnePerchee',
  titre="Quelques îlots dans la mer des grandes cultures de la limagne 'perchée'",
  lieu="Nord-ouest de Bens, Artonne, Puy-de-Dôme", altitude=480,
  date="7 juin 2020", heure="10h45", duree="1 mn",
  auteur="<a href=\"http://jpmeuret.free.fr/nature.html\" target=\"_blank\">Jean-Philippe Meuret</a>",
  licence="<a href=\"https://creativecommons.org/licenses/by-nc-sa/4.0/deed.fr\" target=\"_blank\">CC BY-NC-SA</a>",
  milieux="""Grandes cultures, haies et broussailles en bordure de plateau calcaire.""",
  fichierSon="200607-1045-360-ArtonneBens-JPM.mp3",
  dossierSons=urlDossierSons, dossierAttache=urlDossierAnnexes,
    
  etapes=dict(
      
    lancement="""
      La matinée est bien avancée déjà en cette fin de printemps , mais quelques acharnés s'acharnent encore,
      toujours un peu les mêmes dans ces grandes cultures vallonnées parfois interrompues par un chemin, un noyer,
      une haie, ou des broussailles, à la faveur d'un rideau agraire ou d'une crête incultivable.
      
      5 espèces chantent, dont 1 plus du tout courante en Auvergne, snif, et une autre abondante, toujours en forme,
      rarement discrète et particulièrement en verve ici ; sans doute une 6ème très loin également.
      
      Et un criquet pas encore très chaud apparemment.""",
          
    indices2="""
      3 espèces de bruants sont présentes ici, et c'est logique en ce lieu de transition :
      * au 1er plan, assez régulier, toutes les 10-15s, bien net et isolé à 0s, et à 15s par ex.,
        le bruant des vignes autrefois, et des noyers en Auvergne (en voie de disparition) ;
        à ne pas confondre avec une 4ème espèce non présente ici, à la phrase assez similaire, avec une dernière
        note traînante et plus basse, mais plus grave et vibrée chez notre espèce,
      * assez loin, mais plutôt régulier, toutes les 7-8s : à 1.5s, 9s, par ex., et plus isolé / audible à 17.5s,
        le bruant des grands espaces agricoles quasiment sans haies ou arbre chez nous,
      * enfin, plus furtivement, à 39.5-41.5s, la 3ème espèce, avec 1 seule phrase, typiquement "simpliste"
        avec sa répétition à l'identique d'une seule note, sans autre fioriture ;
        le bruant des vergers "à l'ancienne", des haies et jardins alentour des villages, et du bocage pas trop arboré,
        le tout bien ensoleillé, voire un peu sec.
      
      Il y a également une fauvette commune, assez ubiquiste en terme de milieux naturels,
      pourvu qu'il y ait au moins qq arbres et haies denses ou ombragées,
      aux phrases assez typiques en 2 temps, par ex. entre 3s et 10s.
      
      Et pour en finir avec les plus bruyants(!), un insectivore élancé et habituellement démonstratif visuellement :
      bien audible seul à 25-27s et 48-50s.
      Malheureusement, ses phrases sont ici tronquées (quelque chose ou quelqu'un doit le déranger ;-),
      on n'en a que le début ... mais imaginez-le poursuivant ses répétitions de notes (d'autres que celles-ci,
      par plateaux descendants) en descendant (forcément) en plané vers son poste habituel ...
      
      Enfin, au loin, il faut monter le volume et tendre l'oreille (mais c'est suffisemment grave pour surnager
      du reste du concert, plus aigu), un turdidé commun : à 16-17s et 31-32s, par ex.
      """,
          
    reponse="""
        """))

HTML(KAoutGrandesCulturesLimagnePerchee.lecteurHtml(urlDossierSons))

In [ ]:
KAoutBocageForetPreBassesCombrailles = quizz.DescripteurExercice(

  index='d',
  id='BocageBoisPreBassesCombrailles',
  titre="Bocage et bois dans les pré-Basses-Combrailles",
  lieu="Sous les Bardons, Artonne, Puy-de-Dôme", altitude=450,
  date="24 mars 2019", heure="8 heures du matin", duree="52 s",
  auteur="<a href=\"http://jpmeuret.free.fr/nature.html\" target=\"_blank\">Jean-Philippe Meuret</a>",
  licence="<a href=\"https://creativecommons.org/licenses/by-nc-sa/4.0/deed.fr\" target=\"_blank\">CC BY-NC-SA</a>",
  milieux="""Bocage et bois sur les pentes d'un vallon frais, dans le nord du Puy-de-Dôme.""",
  fichierSon="190324-0814-116-ArtonneSousLesBardons-JPM.mp3",
  dossierSons=urlDossierSons, dossierAttache=urlDossierAnnexes,
    
  etapes=dict(
      
    lancement="""
      Et pour ce dernier concert, plus chargé, retour en tout début de printemps : 1 seul migrateur est arrivé ici.
      
      Un grand bois de vieux chênes couvre la pente de l'autre côté du vallon ; un ruisseau coule en bas,
      et il y a 1 étang pas très loin ; la ferme la plus proche est à plus de 500m, hors de portée auditive.
      
      11 espèces très communes (ah non, 12 finalement, au moins :-).  
      """,
          
    indices2="""
      Commençons par les 4 espèces aux chants / cris les plus graves, normalement les plus faciles à détecter
      dans le fatras continu de ce concert :
      * un colombidé commun au début (0 à 9s), et plus tard aussi (chant),
      * un corvidé commun plus tard, entre 28 et 31s (3 cris "crâa"),
      * un rapace commun (2 individus en fait), entre 32s et 42s (cris de parade ?),
      * un grand échassier en vol à 16s, 26s, 40s et 50s (cris courts et forts qui résonnent).
    
      Ensuite, concentrons-nous sur les espèces dont le chant ressort d'une autre manière de ce fatras
      (soit parce qu'il est plus aigu que la moyenne, soit parce qu'il se trouve isolé dans une relative pause
       du concert, soit parce qu'il est répété en rythme, ... ou tout à la fois :-) :
      * vers 10-12s, ou 14-16s (par ex.) : une mésange au chant simple, aigu, rythmé, puissant,
      * entre 27s et 30s (mais aussi 18-20s, 37-39s, ...) : un "tuii" ou "puii" ou "cuii" répété bien en rythme,
        d'une espèce forestière particulièrement douée pour la "grimpe" (sans filet ou fil de sécurité),
      * entre 47s et 50s, une drôle et jolie sonnerie de portable, répétée 4 ou 5 fois : un turdidé mélomane
        (c'est encore lui - puisque seuls les mâles chantent - les répétitions entre 6s et 8s,
         ou encore - moins sonores - mais doublement répétées, entre 44s et 46s),
      * plus difficile, car plus éloigné, mais son chant porte loin, et dans la bande de fréquence "médiane" :
        entre 8.5s et 10s (bien isolé), ou encore entre 15 et 17s par ex. (plus fort, plus long, mais un peu moins isolé),
        on retrouve le fringille forestier "de service" du parc du chateau de St-Agoulin (enfin ... un autre individu ;-),
      * pas facile, masqué par le colombidé et le fringille, pas très fort, mais bien rythmé, bien aigu
        (c'est pourquoi il ressort) : il ne fait qu'une apparition a priori, juste entre 0.5-3.5s ...
        un petit insectivore commun, modeste et discret, mais rapide et agile (chant).
        
      Enfin (ou presque), une espèce qui chante presque sans interruption tout le long de l'enregistrement
      (et pas très loin, ou en lisière bien sonore), mais difficile à identifier car son chant évolue constamment
      entre les sur-aigus et les aigus presque, dans de longues notes modulées, oscillantes et descendantes par plateaux ;
      les sur-aigus surnagent assez nettement au dessus du concert, mais pas les aigus !
      * ex. de suraigus dans des pauses relatives du concert : 17s, 23s, 28s,
      * entre 17s et 18s, on entend bien, après le plateau sur-aigu, le plateau suivant plus "grave" juste avant 18s
        et le début d'une phrase de la sittelle,
      * de même à 51s, avec un plateau sur-aigu d'abord, puis un plateau très oscillant dans les médium,
        euh non, pardon, dans les aigus ... à la hauteur du chant de la charbonnière !
      * (sur-aigu = 8 kHz, hauteur des roitelets / grimpereaux, charbonnière = 4 kHz, 2 fois moins aigus :-)
        
      Restent 2 furtifs pas évidents :
      * 2 séries de cris "tchac" assez graves, répétés, entre 18s et 23s, d'un turdidé commun,
      * 3 séries de cris "tsi" typiques d'une autre mésange, à 42s et 43.5-44s, et aussi 18-19s.
      
      Et j'en ai peut-être ratés ...
      """,
          
    reponse="""
        """))

HTML(KAoutBocageForetPreBassesCombrailles.lecteurHtml(urlDossierSons))

# Génération de la page

In [ ]:
KQuizz = \
[quizz.DescripteurQuizz(id='Aout', titre='Quizz d\'août',
                        intro=KAoutIntro,
                        publier=dict(lancement=True, indices2=True, reponse=False),
                        exercices=[KAoutBocagePrairiesMoyenneAltitude, KAoutVergerVillageChateau,
                                   KAoutGrandesCulturesLimagnePerchee, KAoutBocageForetPreBassesCombrailles],
                        anecdotes=[],
                        effort=dict(infrastructure=1, lancement=7, indices2=4, reponse=2, anecdotes=0, courrier=5)),
]

nomFicCible = \
    quizz.buildHtmlPage(titre="Les Oiseaux à l'Oreille : Quizzzzz 2020",
                        sousTitre="C'est la reprise, fini les vacances : à vos oreilles, prêt(e)s, écoutez !",
                        description="Quizz 2020 d'application de la formation \"Les Oiseaux à l'Oreille\"",
                        motsClef='auvergne, forêt, bocage, montagne, plaine, '
                                 'bruant, fauvette, alouette, pouillot, pipit',
                        preambule=KPreambule, quizz=KQuizz, etapes=KEtapes, attribsEtMercis=KAttribsEtMercis,
                        dossierAttache=urlDossierAnnexes, dossierSons=urlDossierSons,
                        images=dict(imgTMat=[dict(img='GriMus-MJLombardy-20130304a.jpg',
                                                  legend='Grive musicienne'),
                                             dict(img='BruOrt2-YLeCorre-201804.jpg',
                                                  legend='Bruant ortolan')]),
                        notebook='Quizz2020.ipynb', prefixeFicCible='quizz-2020',
                        effort=sum(sum(q.effort.values()) for q in KQuizz))

HTML("""<p>Quizz 2020 <a href='{nomFic}' target="_blank">{nomFic}</a> (nouvel onglet)</p>""".format(nomFic=nomFicCible))

In [ ]:
raise Exception("On s'arrête ici !")

In [ ]:
implib.reload(module=quizz)

In [ ]:
HTML(quizz.infosEnregXC(nr=471826))